# Imports

In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import math

In [3]:
users = pd.read_csv("datas/users.csv")
users_count = users.shape[0]

users

,Num_Acc,place,catu,grav,sex,trip,secu,locp,actp,etatp,an_nais,num_veh
0,201600000001,1.0,1,1,2,0.0,11.0,0.0,0.0,0.0,1983.0,B02
1,201600000001,1.0,1,3,1,9.0,21.0,0.0,0.0,0.0,2001.0,A01
2,201600000002,1.0,1,3,1,5.0,11.0,0.0,0.0,0.0,1960.0,A01
3,201600000002,2.0,2,3,1,0.0,11.0,0.0,0.0,0.0,2000.0,A01
4,201600000002,3.0,2,3,2,0.0,11.0,0.0,0.0,0.0,1962.0,A01
...,...,...,...,...,...,...,...,...,...,...,...,...
1876000,200500087953,1.0,1,1,1,0.0,13.0,0.0,0.0,0.0,1972.0,B01
1876001,200500087953,1.0,1,1,1,0.0,13.0,0.0,0.0,0.0,1965.0,C01
1876002,200500087953,1.0,1,4,1,5.0,23.0,0.0,0.0,0.0,1990.0,A01
1876003,200500087954,1.0,1,4,1,0.0,13.0,0.0,0.0,0.0,1951.0,A01


In [4]:
users.loc[users.catu == 1, "catu"] = "Driver"
users.loc[users.catu == 2, "catu"] = "Passenger"
users.loc[users.catu == 3, "catu"] = "Pedestrian"
users.loc[users.catu == 4, "catu"] = "Pedestrian in rollerblade or scooter"

users.loc[users.sex == 1, "sex"] = "Male"
users.loc[users.sex == 2, "sex"] = "Female"

users.loc[users.grav == 1, "grav"] = "Unscathed"
users.loc[users.grav == 2, "grav"] = "Killed"
users.loc[users.grav == 3, "grav"] = "Hospitalized wounded"
users.loc[users.grav == 4, "grav"] = "Light injury"

users.loc[users.trip == 1, "trip"] = "Home - work"
users.loc[users.trip == 2, "trip"] = "Home - school"
users.loc[users.trip == 3, "trip"] = "Shopping - Shopping"
users.loc[users.trip == 4, "trip"] = "Professional use"
users.loc[users.trip == 5, "trip"] = "Promenade - leisure"
users.loc[users.trip == 9, "trip"] = "Other"
users.loc[users.trip == 0, "trip"] = "Unknown"

def secu(x):
    if x == "1":
        return "Belt"
    elif x == "2":
        return "Helmet"
    elif x == "3":
        return "Children's device"
    elif x == "4":
        return "Reflective equipment"
    else:
        return "Other"

users["has_secu"] = users.secu.apply(lambda x: True if str(x)[1] == "1" else False)
users["secu"] = users.secu.apply(lambda x: secu(str(x)[0]))

users

,Num_Acc,place,catu,grav,sex,trip,secu,locp,actp,etatp,an_nais,num_veh,has_secu
0,201600000001,1.0,Driver,Unscathed,Female,Unknown,Belt,0.0,0.0,0.0,1983.0,B02,True
1,201600000001,1.0,Driver,Hospitalized wounded,Male,Other,Helmet,0.0,0.0,0.0,2001.0,A01,True
2,201600000002,1.0,Driver,Hospitalized wounded,Male,Promenade - leisure,Belt,0.0,0.0,0.0,1960.0,A01,True
3,201600000002,2.0,Passenger,Hospitalized wounded,Male,Unknown,Belt,0.0,0.0,0.0,2000.0,A01,True
4,201600000002,3.0,Passenger,Hospitalized wounded,Female,Unknown,Belt,0.0,0.0,0.0,1962.0,A01,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1876000,200500087953,1.0,Driver,Unscathed,Male,Unknown,Belt,0.0,0.0,0.0,1972.0,B01,False
1876001,200500087953,1.0,Driver,Unscathed,Male,Unknown,Belt,0.0,0.0,0.0,1965.0,C01,False
1876002,200500087953,1.0,Driver,Light injury,Male,Promenade - leisure,Helmet,0.0,0.0,0.0,1990.0,A01,False
1876003,200500087954,1.0,Driver,Light injury,Male,Unknown,Belt,0.0,0.0,0.0,1951.0,A01,False


In [5]:
drivers = users[users.catu == "Driver"]
drivers_count = len(drivers)

In [6]:
def bar(df, x, labels):
    return px.bar(
        x=x,
        y=df * 100,
        text=(df * 100).apply(lambda x: str(math.floor(x)) + "%"),
        range_y=[0, 100],
        labels=labels
    )

def pie(df, column, title=None):
    data = df[column].value_counts() / len(df)
    
    return px.pie(
        data,
        title=column if title == None else title,
        values=data,
        names=data.index
    )

In [7]:
pies = dict(
    catu=pie(users, "catu", "User categorie"),
    sex=pie(drivers, "sex", "Sex by accident"),
    trip=pie(drivers, "trip", "Trip categorie"),
    grav=pie(users, "grav", "Gravity of accidents"),
    has_secu=pie(users, "has_secu", "Had security"),
    secu=pie(users, "secu", "Categorie type"),
)

# Catégorie d'utilisateur lors d'un accident

In [8]:
pies["catu"]

# Sexe des concernés par un accident

In [9]:
pies["sex"]

# Voyage effectué

In [10]:
pies["trip"]

# Gravité de l'accident

In [11]:
pies["grav"]

In [12]:
pies["secu"]